In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import math

In [2]:
url = 'https://www.futfanatics.com.br/loja/catalogo.php?loja=311840&categoria=954&variacao=tamanho_41&range=&order=2&pg=1'

headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"}

site = requests.get(url, headers=headers)

encoding = site.encoding if 'charset' in site.headers.get('content-type', '').lower() else None
parser = 'html5lib'
soup = BeautifulSoup(site.content, parser, from_encoding=encoding)

totais = soup.find_all('div', class_="search-counter")

total_texto = totais[0].get_text().strip()

index = total_texto.find(' ')

qtd  = total_texto[:index]

ultima_pagina = math.ceil(int(qtd)/40)

print("Quantidade de produtos:",qtd)
print("Quantidade de páginas:",ultima_pagina)

Quantidade de produtos: 521
Quantidade de páginas: 14


In [3]:
dic_produtos = {'marca':[], 'preco':[], 'desconto':[], 'link':[]}

for i in range(1, ultima_pagina+1):
    print('Página', i, 'de', ultima_pagina)
    url_pag = f'https://www.futfanatics.com.br/loja/catalogo.php?loja=311840&categoria=954&variacao=tamanho_41&range=&order=2&pg={i}'
    site = requests.get(url_pag, headers=headers)
    soup = BeautifulSoup(site.content, parser, from_encoding=encoding)
    produtos = soup.find_all('div', class_=re.compile('product-item'))

    for produto in produtos:
        marca = produto.find('div', class_=re.compile('product-name')).get_text().strip()
        preco = produto.find('div', class_='price').get_text().strip()
        desconto = 0
        descontos = produto.find_all('span', class_=re.compile('discount'))
        if len(descontos) > 0:
            desconto = descontos[0].get_text().strip()
        link = produto.find_all('a', href=True)[0]['href']
        dic_produtos['marca'].append(marca)
        dic_produtos['preco'].append(preco)
        dic_produtos['desconto'].append(desconto)
        dic_produtos['link'].append(link)

df = pd.DataFrame(dic_produtos)

Página 1 de 14
Página 2 de 14
Página 3 de 14
Página 4 de 14
Página 5 de 14
Página 6 de 14
Página 7 de 14
Página 8 de 14
Página 9 de 14
Página 10 de 14
Página 11 de 14
Página 12 de 14
Página 13 de 14
Página 14 de 14


In [4]:
df.head()

,marca,preco,desconto,link
0,Tenis New Balance Fresh Foam X 1080 Unlaced Prata,"R$1.299,90",0,https://www.futfanatics.com.br/tenis-new-balan...
1,Tenis New Balance Fresh Foam X 1080 Unlaced Br...,"R$1.299,90",0,https://www.futfanatics.com.br/tenis-new-balan...
2,Tênis New Balance Fresh Foam X 1080 Unlaced Preto,"R$1.299,90",0,https://www.futfanatics.com.br/tenis-new-balan...
3,Tênis New Balance More v4 Azul,"R$1.249,90",0,https://www.futfanatics.com.br/tenis-new-balan...
4,Tênis New Balance 1080 V12 London Edition Preto,"R$1.249,90",0,https://www.futfanatics.com.br/tenis-new-balan...


In [5]:
tabela = df.copy()

tabela['preco'] = tabela['preco'].apply(lambda x: str(x).replace(".",'').replace("R$",''))
tabela['preco'] = pd.to_numeric(tabela['preco'].apply(lambda x: str(x).replace(",",'.')))
tabela['desconto'] = pd.to_numeric(tabela['desconto'].apply(lambda x: str(x).replace("%",'')))

tabela = tabela.sort_values(by=['desconto'],ascending=False)

tabela.head(15)

,marca,preco,desconto,link
143,Tênis Under Armour HOVR Infinite 2 Coral e Preto,499.9,44,https://www.futfanatics.com.br/tenis-under-arm...
483,Tênis Puma Wired Run BDP Azul Marinho,219.9,42,https://www.futfanatics.com.br/tenis-puma-wire...
480,Tênis Fila Racer One Marinho,229.9,42,https://www.futfanatics.com.br/tenis-fila-race...
213,Tênis Under Armour HOVR Summit 2 Vermelho,399.9,42,https://www.futfanatics.com.br/tenis-under-arm...
407,Tênis Olympikus Corre 1 Eco Bege e Marrom,289.9,42,https://www.futfanatics.com.br/tenis-olympikus...
426,Tênis Puma Aviator ENG Pro BDP Azul,279.9,41,https://www.futfanatics.com.br/tenis-puma-avia...
439,Tênis Puma Flyer Runner Speckless BDP Marinho,269.9,40,https://www.futfanatics.com.br/tenis-puma-flye...
510,Tênis Olympikus Metrópole Marinho e Cinza,179.9,40,https://www.futfanatics.com.br/tenis-olympikus...
479,Tênis Puma Accent NXT BDP Marinho e Amarelo,229.9,39,https://www.futfanatics.com.br/tenis-puma-acce...
476,Tênis Puma Flyer Runner BDP Preto,229.9,39,https://www.futfanatics.com.br/tenis-puma-flye...


In [6]:
tabela.to_csv('precoTenis.csv', encoding='utf-8', sep=',', index=False)